In [2]:
import cv2
import torch
import math
import numpy as np
from collections import deque
from ultralytics import YOLO

class SyringeVolumeCalculator:
    def __init__(self):
        self.model = YOLO("runs/pose/train-pose11x-v12-unfinished/weights/best.pt").eval()
        self.device = "mps" if torch.backends.mps.is_available() else "cpu"
        self.model.to(self.device)
        self.volume_history = deque(maxlen=15)
        self.syringe_diameter = 1.0  # cm

    def initialize_camera(self):
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        if not cap.isOpened():
            raise IOError("Camera initialization failed")
        return cap

    @staticmethod
    def calculate_line_equation(p1, p2):
        """Returns (A, B, C) coefficients for line equation Ax + By + C = 0"""
        x1, y1 = p1
        x2, y2 = p2
        a = y2 - y1
        b = x1 - x2
        c = (x2 - x1)*y1 - (y2 - y1)*x1
        return (a, b, c)

    def calculate_parallel_distance(self, line1, line2):
        """Calculate distance between two parallel lines in pixels"""
        a1, b1, c1 = line1
        a2, b2, c2 = line2
        
        # Verify lines are parallel
        if abs(a1*b2 - a2*b1) > 1e-6:
            return None
        
        return abs(c2 - c1) / math.hypot(a1, b1)

    def calculate_height(self, lower_line_eq, upper_line_eq, scale_factor):
        """Calculate height using multiple methods and return consensus"""
        # Method 1: Parallel line distance
        h_pixels = self.calculate_parallel_distance(lower_line_eq, upper_line_eq)
        
        # Method 2: Average point distances (requires physical points)
        try:
            d_ll = self.point_to_line_distance(self.ll_point, upper_line_eq)
            d_lr = self.point_to_line_distance(self.lr_point, upper_line_eq)
            avg_point_dist = (d_ll + d_lr) / 2
        except AttributeError:
            avg_point_dist = h_pixels if h_pixels else 0

        if h_pixels is None or abs(h_pixels - avg_point_dist) > 5:
            return avg_point_dist * scale_factor
        
        return ((h_pixels * 0.7) + (avg_point_dist * 0.3)) * scale_factor

    @staticmethod
    def point_to_line_distance(point, line):
        """Calculate perpendicular distance from point to line"""
        x, y = point
        a, b, c = line
        return abs(a*x + b*y + c) / math.hypot(a, b)

    def process_frame(self, frame):
        results = self.model.predict(frame, verbose=False, conf=0.6)
        if not results:
            return frame, None
        
        r = results[0]
        annotated_frame = r.plot()
        
        if r.keypoints is None or len(r.keypoints.xy[0]) < 4:
            return annotated_frame, None

        kpts = r.keypoints.xy[0].cpu().numpy()
        self.ll_point, self.ul_point, self.ur_point, self.lr_point = kpts[:4]

        # Draw debug information
        self.draw_debug_info(annotated_frame)

        try:
            # Calculate line equations from physical points
            lower_line_eq = self.calculate_line_equation(self.ll_point, self.lr_point)
            upper_line_eq = self.calculate_line_equation(self.ul_point, self.ur_point)

            # Calculate scale factors from both edges
            lower_edge_px = np.linalg.norm(self.lr_point - self.ll_point)
            upper_edge_px = np.linalg.norm(self.ur_point - self.ul_point)
            scale_factor = 2.0 / (lower_edge_px + upper_edge_px)

            # Calculate height using consensus method
            height_cm = self.calculate_height(lower_line_eq, upper_line_eq, scale_factor)
            
            if 0 < height_cm < 30:  # Physical constraint check
                volume = math.pi * (self.syringe_diameter/2)**2 * height_cm
                self.volume_history.append(volume)
                return annotated_frame, np.median(self.volume_history)
            
        except Exception as e:
            print(f"Processing error: {str(e)}")
        
        return annotated_frame, None

    def draw_debug_info(self, frame):
        """Visual debug information overlay"""
        debug_params = {
            'fontFace': cv2.FONT_HERSHEY_SIMPLEX,
            'fontScale': 0.6,
            'color': (0, 0, 255),
            'thickness': 1
        }

        # Keypoint labels
        for label, pt in zip(['LL', 'UL', 'UR', 'LR'], 
                           [self.ll_point, self.ul_point, self.ur_point, self.lr_point]):
            cv2.putText(frame, label, 
                       (int(pt[0])+10, int(pt[1])), 
                       **debug_params)

        # Edge length display
        lower_len = np.linalg.norm(self.lr_point - self.ll_point)
        upper_len = np.linalg.norm(self.ur_point - self.ul_point)
        cv2.putText(frame, f"Lower: {lower_len:.1f}px", (10, 100), **debug_params)
        cv2.putText(frame, f"Upper: {upper_len:.1f}px", (10, 130), **debug_params)

    def run(self):
        cap = self.initialize_camera()
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                processed_frame, volume = self.process_frame(frame)
                
                if volume is not None:
                    cv2.putText(processed_frame, f"Volume: {volume:.2f} mL", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                               1, (0, 255, 0), 2)
                    cv2.putText(processed_frame, f"Height: {volume / (math.pi*0.25):.1f} cm", 
                               (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.8, (0, 255, 0), 2)

                cv2.imshow('Syringe Volume Measurement', processed_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            cap.release()
            cv2.destroyAllWindows()

if __name__ == "__main__":
    calculator = SyringeVolumeCalculator()
    calculator.run()


2025-01-25 14:01:18.189 python[38341:1072831] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-25 14:01:18.189 python[38341:1072831] +[IMKInputSession subclass]: chose IMKInputSession_Modern
/var/folders/ht/hpy25npj1t96b4yb_vxvc_b00000gn/T/ipykernel_38341/3429443511.py:43: RuntimeWarning: invalid value encountered in divide
  return abs(c2 - c1) / math.hypot(a1, b1)
